# Urban Demography - Initial Zonal Data Extraction

- Author: Andrew Zimmer 
- Date Created: 2024-11-14 
- Last Edited:  2024-11-14 
- Version: 1.0

Description: 
- This script processes raster files representing population data for different age and sex groups, calculates zonal statistics (sum) using a shapefile for urban boundaries, and saves the results as CSV files. 
- The processing is parallelized for faster computation, and the results are saved yearly in separate CSV files. The correct shapefile for each year is dynamically selected based on a predefined mapping.

Input Files:
- Raster files in the directory: './data/raw/worldpop_agesex/' representing population data in GeoTIFF format. These are available for download on the [WorldPop](https://www.worldpop.org/) site
- Shapefiles in the directory: './Data/Urban-Boundaries/Shapefiles/' corresponding to urban boundaries for specific years. These are available for download on the [GHS-UCDB](https://human-settlement.emergency.copernicus.eu/ghs_ucdb_2024.php) site.

Output Files:
- Separate CSV files for each year saved in the directory: './01_data/urban_agesex_output/'
- CSV Filename format: 'population_zonal_stats_{year}.csv'

Steps in Script
- Reads in raster files for different years and processes them in parallel to calculate zonal statistics.
- Extracts zonal statistics (sum) for each urban center and age-sex group.
- Saves the results in separate CSV files for each year.

In [ ]:
import os
import geopandas as gpd
import rasterio
import rasterstats
import pandas as pd
from multiprocessing import Pool

def process_file(file_info):
    """Processes a single file and returns zonal statistics results."""
    filename, input_folder, shapefile_dir, year_shapefile_map = file_info
    file_path = os.path.join(input_folder, filename)
    
    # Extract year, gender, and age from filename
    parts = filename.split('_')
    gender = parts[1]  # 'f' or 'm'
    age = parts[2]
    year = int(parts[3])  # Extract year as integer

    # Determine the correct shapefile based on the year
    shapefile_name = None
    for year_range, shapefile in year_shapefile_map.items():
        if year in year_range:
            shapefile_name = shapefile
            break

    if not shapefile_name:
        print(f"No matching shapefile found for year {year}")
        return []

    print(f"Processing file: {filename} for year {year}, age {age}, sex {gender} using shapefile: {shapefile_name}")

    # Load the appropriate polygons
    polygons = gpd.read_file(os.path.join(shapefile_dir, shapefile_name))

    # Ensure CRS matches between polygons and raster (reproject if necessary)
    with rasterio.open(file_path) as src:
        raster_crs = src.crs  # Expected CRS is WGS 84
        if polygons.crs != raster_crs:
            print(f"Reprojecting polygons from {polygons.crs} to {raster_crs}")
            polygons = polygons.to_crs(raster_crs)

    # Calculate zonal statistics, focusing only on sum
    results = []
    with rasterio.open(file_path) as src:
        stats = rasterstats.zonal_stats(
            polygons,
            src.read(1, masked=True),  # Read only masked areas to save memory
            affine=src.transform,
            stats=['sum'],
            nodata=src.nodata
        )

        # Collect results with appropriate metadata
        for i, stat in enumerate(stats):
            results.append({
                'ID_UC_G0': polygons.iloc[i]['ID_UC_G0'],  # Use the correct ID field
                'year': year,
                'age': age,
                'sex': 'female' if gender == 'f' else 'male',
                'sum': stat['sum']
            })
    return results

def extract_info_to_csv(input_folder, output_folder, shapefile_dir, year_shapefile_map, num_workers=20):
    """
    Extracts zonal statistics from raster files in parallel and saves results to separate yearly CSV files.
    
    Parameters:
        input_folder (str): Path to the folder containing the raster files.
        output_folder (str): Folder path to save the resulting CSV files for each year.
        shapefile_dir (str): Directory containing the reprojected shapefiles.
        year_shapefile_map (dict): Dictionary mapping year ranges to shapefile names.
        num_workers (int): Number of CPU cores to use for parallel processing.
    """
    
    # List and sort files by year
    files = sorted([f for f in os.listdir(input_folder) if f.endswith('.tif')], key=lambda x: int(x.split('_')[3]))

    # Group files by year for sequential processing of each year
    years = sorted(set(int(f.split('_')[3]) for f in files))
    for year in years:
        # Filter files for the current year
        year_files = [f for f in files if int(f.split('_')[3]) == year]
        args = [(f, input_folder, shapefile_dir, year_shapefile_map) for f in year_files]

        # Collect results for the current year
        year_results = []
        with Pool(processes=num_workers) as pool:
            for file_results in pool.imap_unordered(process_file, args):
                if file_results:
                    year_results.extend(file_results)
        
        # Save the current year's results to a CSV file
        os.makedirs(output_folder, exist_ok=True)
        year_df = pd.DataFrame(year_results)
        output_csv = os.path.join(output_folder, f"population_zonal_stats_{year}.csv")
        year_df.to_csv(output_csv, index=False)
        print(f"Zonal statistics for {year} saved to {output_csv}")

# Set input parameters here
input_folder = './01_data/00_worldpop_agesex/'          # Folder containing the .tif files
output_folder = './01_data/01_urban_agesex_output/'     # Folder to save the resulting CSV files
shapefile_dir = './01_data/03_ghs_ucdb/'                # Directory with reprojected shapefiles

# Dictionary mapping year ranges to corresponding shapefile names
# this is changed out if you are only interested in a static boundary. For this - you can use the GHS_UCDB_MTUC_2020_WGS84.shp
# it represents the maximum extent of all urban areas across all years.
year_shapefile_map = {
    range(2000, 2003): 'GHS-UCDB-MTUC-2000-WGS84.shp',
    range(2003, 2008): 'GHS-UCDB-MTUC-2005-WGS84.shp',
    range(2008, 2013): 'GHS-UCDB-MTUC-2010-WGS84.shp',
    range(2013, 2018): 'GHS-UCDB-MTUC-2015-WGS84.shp',
    range(2018, 2021): 'GHS-UCDB-MTUC-2020-WGS84.shp'
}

# Run the function
extract_info_to_csv(input_folder, output_folder, shapefile_dir, year_shapefile_map, num_workers=20)

Processing file: global_f_10_2000_1km.tif for year 2000, age 10, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_20_2000_1km.tif for year 2000, age 20, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_40_2000_1km.tif for year 2000, age 40, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_70_2000_1km.tif for year 2000, age 70, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_80_2000_1km.tif for year 2000, age 80, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_75_2000_1km.tif for year 2000, age 75, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_65_2000_1km.tif for year 2000, age 65, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_35_2000_1km.tif for year 2000, age 35, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_05_2000_1km.tif for year 2000, age 05, sex m u

/home/p18q126/.local/lib/python3.9/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


Zonal statistics for 2001 saved to /home/p18q126/myFolder/Projects/Urban-Demography/Data/Urban-AgeSex-Output/population_zonal_stats_2001.csv
Processing file: global_m_40_2002_1km.tif for year 2002, age 40, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_01_2002_1km.tif for year 2002, age 01, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_15_2002_1km.tif for year 2002, age 15, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_80_2002_1km.tif for year 2002, age 80, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_55_2002_1km.tif for year 2002, age 55, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_f_20_2002_1km.tif for year 2002, age 20, sex f using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_35_2002_1km.tif for year 2002, age 35, sex m using shapefile: GHS-UCDB-MTUC-2000-WGS84.shpProcessing file: global_m_45_2002_1km.tif for y